# Magic Formula

In [2]:
import numpy as np
import pandas as pd
import string
import warnings
warnings.filterwarnings('ignore')
import requests

In [3]:
url = 'http://www.fundamentus.com.br/resultado.php'

In [4]:
header = {
  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"  
 }

r = requests.get(url, headers=header)

In [5]:
#na tabela do site fundamentus o decimal é colocado como virgula
df = pd.read_html(r.text,  decimal=',', thousands='.')[0] 

In [6]:
df.columns

Index(['Papel', 'Cotação', 'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo',
       'P/Cap.Giro', 'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA',
       'Mrg Ebit', 'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses',
       'Patrim. Líq', 'Dív.Brut/ Patrim.', 'Cresc. Rec.5a'],
      dtype='object')

In [7]:
#Os indicadores com % são raspados como strings
#Este for transforma eles em float
for coluna in ['Div.Yield', 'Mrg Ebit', 'Mrg. Líq.', 'ROIC', 'ROE', 'Cresc. Rec.5a']:
    df[coluna] = df[coluna].str.replace('.', '')
    df[coluna] = df[coluna].str.replace(',', '.')
    df[coluna] = df[coluna].str.rstrip('%').astype('float') / 100

## Analisando os dados 

In [8]:
#liquidez diária 
df = df[df['Liq.2meses'] > 100000]
df.shape                   

(300, 21)

## Joel Greenblat

In [9]:
df[:5]

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
12,MEGA3,8.61,-486.72,1.04,2.200,0.0,0.318,-39.42,10.21,-0.61,...,13.25,0.2155,-0.0033,0.96,0.0346,-0.0021,17207600.0,5.151090e+09,1.65,9.2252
18,HBSA3,2.46,-229.15,1.41,1.058,0.0,0.294,3.13,6.42,-0.50,...,8.78,0.1649,-0.0046,1.86,0.0536,-0.0061,10973800.0,1.330800e+09,3.42,0.1952
29,SMFT3,14.21,-93.15,1.99,7.707,0.0,0.691,4.71,-76.92,-1.93,...,11.12,-0.1002,-0.0827,2.01,-0.0122,-0.0214,30966300.0,4.186390e+09,0.82,-0.0287
40,BRKM3,20.10,-47.74,2.19,0.166,0.0,0.182,1.56,2.62,-0.34,...,3.40,0.0633,-0.0085,1.42,0.1001,-0.0458,279629.0,7.321800e+09,4.86,0.1878
43,BRKM5,19.35,-45.95,2.11,0.160,0.0,0.175,1.50,2.52,-0.32,...,3.34,0.0633,-0.0085,1.42,0.1001,-0.0458,78312100.0,7.321800e+09,4.86,0.1878


In [20]:
data = df
data = data[data['ROE'] > 0.0]
data = data[data['P/L'] > 3.0]
data = data[data['Mrg. Líq.'] > 0.10]
data = data[data['ROIC'] > 0.0]
#data = data[data['EV/EBITDA'] > 0.0]

In [21]:
print(data.shape, df.shape) 

(101, 21) (300, 21)


In [23]:
num_acoes = data.shape[0]

## Rankings

In [24]:
RankingROE = pd.DataFrame()
RankingROE['pos_roe'] = range(1,num_acoes)
RankingPL = pd.DataFrame()
RankingPL['pos_pl'] = range(1,num_acoes)

In [25]:
RankingROE['Papel'] = data.sort_values(by=['ROE'], ascending=False)['Papel'][:(num_acoes-1)].values
RankingROE['ROE Value'] = data.sort_values(by=['ROE'], ascending=False)['ROE'][:(num_acoes-1)].values

In [26]:
RankingPL['Papel'] = data.sort_values(by=['P/L'], ascending=True)['Papel'][:(num_acoes-1)].values
RankingPL['P/L Value'] = data.sort_values(by=['P/L'], ascending=True)['P/L'][:(num_acoes-1)].values

In [27]:
RankingPL[:5]

,pos_pl,Papel,P/L Value
0,1,GOAU3,3.06
1,2,EUCA4,3.18
2,3,GOAU4,3.28
3,4,GGBR3,3.50
4,5,VALE3,3.53


In [28]:
RankingROE[:5]

,pos_roe,Papel,ROE Value
0,1,CGAS5,1.7540
1,2,KEPL3,0.6404
2,3,UNIP6,0.5722
3,4,UNIP3,0.5722
4,5,VALE3,0.5127


## Soma das posições - Tabela das ações mais descontadas

In [42]:
ranking = pd.merge(RankingPL, RankingROE)

In [43]:
ranking['pts'] = ranking["pos_pl"] + ranking["pos_roe"]
ranking[:3]

,pos_pl,Papel,P/L Value,pos_roe,ROE Value,pts
0,1,GOAU3,3.06,34,0.2275,35
1,2,EUCA4,3.18,78,0.1279,80
2,3,GOAU4,3.28,33,0.2275,36


In [44]:
rank = ranking.sort_values('pts')
rank = rank[["Papel","P/L Value","pos_pl","ROE Value","pos_roe","pts"]]
rank.set_index("Papel",inplace=True)
rank.reset_index(inplace=True)
rank.index = rank.index + 1
rank.head(5)

,Papel,P/L Value,pos_pl,ROE Value,pos_roe,pts
1,KEPL3,3.86,6,0.6404,2,8
2,VALE3,3.53,5,0.5127,5,10
3,TASA4,3.98,10,0.5077,6,16
4,TASA3,4.02,11,0.5077,7,18
5,KLBN3,4.44,18,0.4908,10,28


## Tabela sem a posição do PL e do ROE

In [86]:
rank = ranking.sort_values('pts')
rank = rank[["Papel","P/L Value","ROE Value","pts"]]
rank.set_index("Papel",inplace=True)
rank.reset_index(inplace=True)
rank.index = rank.index + 1
rank.head(30)

,Papel,P/L Value,ROE Value,pts
1,KEPL3,3.86,0.6404,8
2,VALE3,3.53,0.5127,10
3,TASA4,3.98,0.5077,16
4,TASA3,4.02,0.5077,18
5,KLBN3,4.44,0.4908,28
6,KLBN11,4.45,0.4908,28
7,KLBN4,4.45,0.4908,28
8,UNIP6,5.37,0.5722,32
9,UNIP3,5.19,0.5722,32
10,FESA4,4.30,0.3495,33


In [111]:
def retirar_duplicados(rank):
    Ticker = rank['Papel'].str.extract(r'([A-Z]{4})')
    rank["Ticker"] = Ticker
    rank.set_index("Ticker",inplace=True)
    rank = rank[~rank.index.duplicated(keep='first')]
    rank.reset_index(inplace=True)
    rank.index = rank.index + 1
    rank.drop('Ticker',axis=1,inplace=True)
    return rank


rank = ranking.sort_values('pts')
rank = rank[["Papel","P/L Value","ROE Value","pts"]]
rank = retirar_duplicados(rank)

rank.head(30)

,Papel,P/L Value,ROE Value,pts
1,KEPL3,3.86,0.6404,8
2,VALE3,3.53,0.5127,10
3,TASA4,3.98,0.5077,16
4,KLBN3,4.44,0.4908,28
5,UNIP6,5.37,0.5722,32
6,FESA4,4.30,0.3495,33
7,GGBR3,3.50,0.2478,34
8,GOAU3,3.06,0.2275,35
9,CAMB3,4.36,0.3299,37
10,RECV3,5.13,0.3027,48
